# Taxa _Churn_ - _Telecom_

Estudo prático de análise de dados de taxa de cancelamento de clientes (com *dataset* do [_Kaggle_](https://www.kaggle.com/blastchar/telco-customer-churn));

---

[Open In Colab](https://colab.research.google.com/drive/1GqrSHQwM4blaBalH53Y8O9IDwVyzNzNQ?usp=sharing)

[Open in Kaggle](https://www.kaggle.com/leonichel/taxa-churn-telecom)

---

[Leonichel Guimarães (PIBITI/CNPq-FA-UEM)](https://github.com/leonichel)

Professora Linnyer Ruiz (orientadora)

---

Referências bibliográficas:

GÉRON, Aurélien. _Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems_. 2. ed. O'Reilly Media, 2019.

---

Manna Team  |  UEM       |     CNPq
:----------:|:----------:|:----------:|
<img src="https://manna.team/_next/static/images/logo2-e283461cfa92b2105bfd67e8e530529e.png" alt="Manna Team" width="200"/> | <img src="https://marcoadp.github.io/WebSiteDIN/img/logo-uem2.svg" alt="UEM" width="200"/> | <img src="https://www.gov.br/cnpq/pt-br/canais_atendimento/identidade-visual/logo_cnpq.svg" alt="CNPq" width="200"/>

## Obtenção de dados

### Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Leitura do banco de dados

In [ ]:
df = pd.read_csv('../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

### Exploração inicial

In [ ]:
df.info()

In [ ]:
df.tail()

In [ ]:
df[(df.TotalCharges == " ")]

### Resultados

* _'customerID'_ não é necessária;
* _'SeniorCitizen'_ está com notação binária '0' e '1', ao contrário dos demais atributos binários; Para contornar esse problema, é necessário mapear os '0' para 'No', e os '1' para 'Yes';
* _'TotalChanges'_ está atribuida com tipo _'object'_, quando deveria estar com tipo _'float64'_; O motivo da ocorrência é pela existência de 11 linhas que contêm o valor nulo '" "' como _string_; Para contornar, é necessário remover essas 11 linhas, e converter a coluna para o tipo numérico.

## Limpeza de dados

### Remover _'customerID'_

In [ ]:
df.drop('customerID', axis=1, inplace=True)

### Alterar valores binários de _'SeniorCitizen'_ para 'Yes' e 'No'

In [ ]:
filter = lambda x: 'Yes' if x == 1 else 'No'
df['SeniorCitizen'] = df['SeniorCitizen'].apply(filter)

### Removendo linhas com valores nulos de _'TotalCharges'_

In [ ]:
df.drop(df[(df.TotalCharges == " ")].index, axis=0, inplace=True)

In [ ]:
df.TotalCharges = pd.to_numeric(df.TotalCharges)

### Verificando resultado

In [ ]:
df.info()

In [ ]:
df.tail()

### Resultados

* _'customerID'_ foi removida;
* _'SeniorCitizen'_ teve seus valores mapeados para a forma padrão;
* _'TotalCharges'_ teve seus valores nulos removidos, e o tipo alterado para numérico.


## Análise de dados

### Explorando contegem dos valores de cada coluna

In [ ]:
for col in df.columns:
    print(df[col].value_counts())
    print()

### Cálculo da taxa de churn global

In [ ]:
n_churn = df['Churn'].value_counts()['Yes']
n_no_churn = df['Churn'].value_counts()['No']

churn_global_rate = n_churn/(n_churn + n_no_churn)
churn_global_rate

### Variáveis que armazenam colunas numéricas e categóricas

In [ ]:
num_cols = df.select_dtypes(exclude=['object']).columns
cat_cols = df.select_dtypes(include=['object']).columns
cat_cols = cat_cols[cat_cols != 'Churn']

cat_cols

### Visualização de dados

#### Configuração de plotes

In [ ]:
sns.set_style('darkgrid')
sns.set_palette('cubehelix')

#### Plotes 1: classes de cada coluna categórica

* Plote de barras;
* Mapeado com quantidade de churns, pela coluna _'Churn'_.

In [ ]:
for col in cat_cols:
    plt.figure(figsize=(10, 6))
    plt.title(col)
    ax = sns.histplot(data=df, x=col, hue='Churn');
    ax.set(xlabel='Values', ylabel='Count')
    plt.show()

#### Plotes 2: taxas de churn das classes de cada coluna categórica

* Plote de barras;
* Taxa de churn em percentual (0 ~ 100%) para cada classe;
* Com marcação horizontal da taxa churn global.

In [ ]:
for col in cat_cols:
    rates = []
    for i in df[col].value_counts().index:
        no_churn = df[df.Churn == 'No'][col].value_counts()[i]
        churn = df[df.Churn == 'Yes'][col].value_counts()[i]
        rate = churn/(churn + no_churn)
        rates.append(rate)
    rates = np.array(rates)*100
    plt.figure(figsize=(10, 6))
    plt.title(col)
    plt.axhline(churn_global_rate*100, ls='--' ,label='global churn rate')
    plt.legend()
    ax = sns.barplot(x=df[col].value_counts().index, y=rates);
    ax.set(xlabel='Values', ylabel='Churn rate')
    plt.show()

#### Plotes 3: classes de cada coluna numérica

* Plote de barras;
* Mapeado com quantidade de churns, pela coluna _'Churn'_.

In [ ]:
for col in num_cols:
    plt.figure(figsize=(10, 6))
    plt.title(col)
    plt.axhline(churn_global_rate, ls='--' , label='global churn rate')
    ax = sns.histplot(data=df, x=col, hue='Churn');
    ax.set(xlabel='Values', ylabel='Count')
    plt.show()

#### Plotes 4: proprorção de taxas de churn das classes de cada coluna numérica

* Plote de barras;
* Taxa de churn em percentual (0 ~ 1) para cada classe;
* Com marcação horizontal da taxa churn global.

In [ ]:
for col in num_cols:
    plt.figure(figsize=(10, 6))
    plt.title(col)
    plt.axhline(churn_global_rate, ls='--' , label='global churn rate')
    ax = sns.histplot(data=df, x=col, hue='Churn', multiple='fill');
    ax.set(xlabel='Values', ylabel='Count')
    plt.show()

#### Plotes 5: correlações de entre colunas numéricas 

* Plote de _heatmap_ e _pairplot_;
* Comparação de coluna a coluna.

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('Correlations of numerical columns')
sns.heatmap(df.corr(), annot=True, cmap=sns.cubehelix_palette(start=.5, rot=-.75, as_cmap=True));

In [ ]:
sns.pairplot(df, hue='Churn');

### Resultados

* A taxa de churn global é 26,58%;

* Os Plotes 1 indicam o perfil majoritário dos clientes:
  * Jovem ou adulto;
  * Sem dependentes (filhos);
  * Com serviço de telefonia;
  * Os demais atributos não apresentam prevalência relevância: gênero, estado civil, tipos de serviços adotados, e forma de contrato e pagamento.

* Atrelado aos Plotes 1, os Plotes 2 indicam o perfil majoritário dos clientes que cancelam:
  * Idosos: a taxa de churn em idosos é acima da global, com um valor superior a 40%;
  * Solteiros: a taxa em solteiros é acima da global, com um valor superior a 35%; os clientes com parceiros (não solteiros) apresentam uma taxa inferior a 20%;
  * Sem dependentes: a taxa em clientes sem dependentes é acima da global, com um valor superior a 30%; os clientes com dependentes apresentam uma taxa inferior a 20%;
  * Com fibra ótica: a taxa em clientes com fibra é acima da global, com um valor superior a 40%; os clientes com as demais opções apresentam taxas inferiores a 20%; 
  * Sem serviços adicionais (segurança e backup online, proteção de dispositivo, suporte técnico): as taxas para clientes que não assinam esses serviços adicionais são superiores a global, com valores que variam entre 30% a 45%;
  * Sem ou com serviços de streaming (TV e filmes): as taxas dos clientes que utilizam e dos clientes que não utilizam são ambas superiores a global; 
  * Com contrato mês-a-mês: a taxa para esses clientes é acima de 45%; os demais apresentam taxas inferiores a 15%;
  * Com fatura virtual: a taxa para esses clientes é acima de 30%; os que não têm apresentam taxa inferior a 20%;
  * Com método de pagamento de cheque eletrônico: a taxa para esses clientes é acima de 40%; as taxas dos demais métodos variam entre 10% a 20%.

* Atrelados aos Plotes 2, os Plotes 3 indicam:
  * Um alto número de clientes com 0 a 5 meses de contrato, e com mais de 65 (5,5 anos); Os demais clientes, com outros intervalos de contrato (5 ~ 65 meses) são bem mais minoritários;
  * Um alto número de clientes que realizam os valores mínimos de recarga;

* Os Plotes 4 revelam:
  * A taxa de churn diminuí significativamente de acordo com o número de meses que os contratos têm; A taxa caí de mais de 55% (0 ~ 5 meses) para um valor inferior a taxa global (20 ~ 25 meses);
  * A taxa churn apresenta queda conforme o total de recargas que os cliente fazem aumentam.

* Os Plotes 5 revelam:
  * Uma alta correlação entre o número de recargas totais e mensais com o tempo de contrato dos clientes.




## Conclusão

* Mais de 1/4 dos clientes são perdidos - ou seja, a empresa perde 1 a cada 4 de seus contratos;
* O cliente típico da empresa é: jovem, sem dependetes, e com serviços de telefonia; 
* O cliente típico que cancela é: idoso, solteiro, sem dependetes, com fibra ótica, sem serviços adicionais, com contrato mês-a-mês, com fatura virtual, e que paga com cheque eletrônico (forma manual); Os clientes estão com fibra ótica estão descontentes;
* Há na empresa um evidente e grave problema de retenção de clientes;
* Os clientes que realizam recargas maiores cancelam menos;

## Recomendações

* Recomenda-se fortemente a adoção de ações que diminuem essa taxa, que certamente aumentará a receita da empresa;
* Recomenda-se ação para identificar o motivo de churn em idosos;
* Recomenda-se ação para identificar o motivo de descontentamento com o serviço de fibra ótica;
* Recomenda-se adoção de políticas de vendas de planos, a fim de insentivar os clientes a adquirirem serviços adicionais;
* Recomenda-se adoção de promoções, que incentivem os clientes a optarem por contratos anuais, por métodos de pagamentos automáticos, e por valores de recargas mais altos;
* Recomenda-se adoção de políticas que melhorem a retenção de clientes;
